In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%autosave 10

import sys, os
sys.setrecursionlimit(10000)

gpu_id='gpu2'
os.environ["THEANO_FLAGS"] = "device="+gpu_id +",lib.cnmem=1"

import warnings
warnings.filterwarnings("ignore")

import numpy as np
from keras.layers.advanced_activations import SReLU
from keras import callbacks as ckbs
import random
import time
import gzip
from keras.models import model_from_json
import pickle as pkl
import keras
from keras.models import Sequential
from keras.layers.core import *
from keras.layers.wrappers import *
from keras.optimizers import *
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
from docutils.languages.af import labels
from sklearn.model_selection import train_test_split

Autosaving every 10 seconds


Using Theano backend.


In [3]:


from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Activation, Dense, Dropout, LSTM, Merge
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 4007)


In [3]:
LEMMA_IND = 7
POS_IND = 9
SELECTED_FEATURES = "/home/sag/syntactic/src_putin/grant/data/processed/selected_features/"
BEST_MODEL = SELECTED_FEATURES+"model_with_w2v_init.pkl"
RESULTS = SELECTED_FEATURES+"results1.pkl"

In [4]:
matrix = np.load(SELECTED_FEATURES+"matrix_updated.npy")
w2v_weights = np.load(SELECTED_FEATURES+"w2v_weights.npy")

target = OneHotEncoder(sparse=False).fit_transform(np.load(SELECTED_FEATURES+"y.npy").reshape(-1, 1) - 1)
target = np.delete(target, (71079), axis=0)
print(matrix.shape, target.shape)

#X_train, X_val, X_test, y_train, y_val, y_test = matrix_train_val_test_split(matrix, target, number_of_samples_to_use=10000)
X_train, X_val, X_test, y_train, y_val, y_test = matrix_train_val_test_split(matrix, target)

((1423937, 13, 13), (1423937, 125))
((1153388, 13, 13), (128155, 13, 13), (142394, 13, 13), (1153388, 125), (128155, 125), (142394, 125))


In [5]:
# params = {
#     'lstm_layers' :[[450, 450, 450], [400, 400, 400]],
#     'dense_layers': [[100, 100], [100]],
#     'loss': ['categorical_crossentropy'],
#     'activation': ['srelu', 'prelu', 'elu'],
#     'optimizer': [Eve(), 'Adam'],
#     'metrics': [f1],
#     'dropout': [0.25]
# }

# fit_params = {
#     'nb_epoch': 100,
#     'batch_size': 3072,
#     'verbose': 2,
#     'callbacks': [EarlyStopping(monitor='val_loss', patience=5)],
#     'validation_data': [X_val, y_val]
# }  

# branches = create_embedding_with_w2v_matrix(matrix, w2v_weights)
# results, best_model, best_score = keras_gridsearch(
#                      create_stacked_lstm, \
#                      branches, \
#                      X_train, y_train, \
#                      X_val, y_val, \
#                      X_test, y_test, \
#                      params, fit_params, \
#                      save_best_model=BEST_MODEL,
#                      save_results=RESULTS, start=0, end=12
#                      )

In [6]:
# model = load_keras_model(BEST_MODEL)
# #results = pickle.load(open(RESULTS, 'rb'))

# stats_val = get_stats(model, X_val, y_val)
# stats_test = get_stats(model, X_test, y_test)
# print(stats_val['f1'])
# print(stats_test['f1'])

In [11]:
param = {
    'lstm_layers' :[450, 450, 450],
    'dense_layers': [100],
    'loss': 'categorical_crossentropy',
    'activation': 'srelu', # 'prelu', 'elu',
    'optimizer': Eve(),
    'metrics': f1,
    'dropout': 0.25
}

fit_params = {
    'nb_epoch': 10,
    'batch_size': 4096,
    'verbose': 2,
    'callbacks': [EarlyStopping(monitor='val_loss', patience=5), 
                  FullModelCheckpoint(BEST_MODEL)],
    'validation_data': [X_val, y_val]
}

branches = create_embedding_with_w2v_matrix(matrix, w2v_weights)

In [12]:
keras_model = KerasClassifier(build_fn=create_stacked_lstm, branches=branches, **param)
keras_model.fit(X_train, y_train, **fit_params)
stats_val = get_stats(keras_model, X_val, y_val)
stats_test = get_stats(keras_model, X_test, y_test)

Train on 1153388 samples, validate on 128155 samples
Epoch 1/10
680s - loss: 1.8677 - f1: 0.3623 - val_loss: 0.9926 - val_f1: 0.6958
Epoch 2/10
680s - loss: 0.7347 - f1: 0.7909 - val_loss: 0.5312 - val_f1: 0.8472
Epoch 3/10
680s - loss: 0.4789 - f1: 0.8633 - val_loss: 0.4429 - val_f1: 0.8721
Epoch 4/10
679s - loss: 0.3664 - f1: 0.8956 - val_loss: 0.4092 - val_f1: 0.8826
Epoch 5/10
679s - loss: 0.2955 - f1: 0.9162 - val_loss: 0.3929 - val_f1: 0.8887
Epoch 6/10
678s - loss: 0.2456 - f1: 0.9304 - val_loss: 0.4028 - val_f1: 0.8915
Epoch 7/10
677s - loss: 0.2085 - f1: 0.9410 - val_loss: 0.4068 - val_f1: 0.8912
Epoch 8/10
679s - loss: 0.1852 - f1: 0.9475 - val_loss: 0.4231 - val_f1: 0.8928
Epoch 9/10
675s - loss: 0.1679 - f1: 0.9525 - val_loss: 0.4367 - val_f1: 0.8922
Epoch 10/10
674s - loss: 0.1566 - f1: 0.9554 - val_loss: 0.4386 - val_f1: 0.8924
142394/142394 [==============================] - 19s    


In [14]:
model = load_keras_model(BEST_MODEL)
#results = pickle.load(open(RESULTS, 'rb'))

stats_val = get_stats(model, X_val, y_val)
stats_test = get_stats(model, X_test, y_test)
print(stats_val['f1'])
print(stats_test['f1'])

142394/142394 [==============================] - 19s    
0.888100784888
0.876953391584
